네이버 기사의 링크, 본문을 추출하는 코드입니다.
아래에서 운영체제에 맞는 chromedriver를 다운받아 같은 폴더에 두시면 됩니다.
https://googlechromelabs.github.io/chrome-for-testing/
주피터 노트북에서 정상 실행됩니다.

In [ ]:
# 기사 링크 저장
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from datetime import datetime, timedelta
from urllib.parse import quote
import time

def get_naver_news_with_selenium(query, start_date, end_date, max_pages=3):
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 창 안 띄우고 실행
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0")

    # ✅ 크롬 드라이버 경로 설정 (본인 환경에 맞게 수정)
    service = Service(executable_path="chromedriver.exe")
    driver = webdriver.Chrome(service=service, options=chrome_options)

    current_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

    results = []

    while current_date <= end_date_dt:
        date_str = current_date.strftime('%Y.%m.%d')
        date_nso = current_date.strftime('%Y%m%d')
        print(f"[{date_str}] 수집 중...")

        for page in range(1, max_pages + 1):
            start = 1 + (page - 1) * 10
            query_encoded = quote(query)
            # sort=0(관련도순), 1(최신순), 2(오래된순)
            url = f"https://search.naver.com/search.naver?where=news&query={query_encoded}&sort=0&ds={date_str}&de={date_str}&nso=so:r,p:from{date_nso}to{date_nso},a:all&start={start}"

            driver.get(url)
            time.sleep(1.5)  # JS 렌더링 기다리기

            # 'n.news.naver.com'이 포함된 모든 a 태그를 선택
            links = driver.find_elements(By.CSS_SELECTOR, "a[href*='n.news.naver.com']")
            if not links:
                break

            for link in links:
                href = link.get_attribute("href")
                if "n.news.naver.com" in href:
                    results.append((query, date_str, href))

        current_date += timedelta(days=1)

    driver.quit()
    return results

# ✅ 실행
news_links = get_naver_news_with_selenium("SPY", "2024-01-01", "2024-12-31") # 키워드, 기간 설정

# ✅ 출력
for row in news_links:
    print(",".join(row))

# ✅ CSV로 저장
import csv
with open("SPY2024_urls.csv", mode="w", newline="", encoding="utf-8-sig") as file:
    writer = csv.writer(file)
    writer.writerow(["company", "date", "url"])  # 헤더 한글로
    writer.writerows(news_links)


[2024.01.01] 수집 중...
[2024.01.02] 수집 중...
[2024.01.03] 수집 중...
[2024.01.04] 수집 중...
[2024.01.05] 수집 중...
[2024.01.06] 수집 중...
[2024.01.07] 수집 중...
[2024.01.08] 수집 중...
[2024.01.09] 수집 중...
[2024.01.10] 수집 중...
[2024.01.11] 수집 중...
[2024.01.12] 수집 중...
[2024.01.13] 수집 중...
[2024.01.14] 수집 중...
[2024.01.15] 수집 중...
[2024.01.16] 수집 중...
[2024.01.17] 수집 중...
[2024.01.18] 수집 중...
[2024.01.19] 수집 중...
[2024.01.20] 수집 중...
[2024.01.21] 수집 중...
[2024.01.22] 수집 중...
[2024.01.23] 수집 중...
[2024.01.24] 수집 중...
[2024.01.25] 수집 중...
[2024.01.26] 수집 중...
[2024.01.27] 수집 중...
[2024.01.28] 수집 중...
[2024.01.29] 수집 중...
[2024.01.30] 수집 중...
[2024.01.31] 수집 중...
[2024.02.01] 수집 중...
[2024.02.02] 수집 중...
[2024.02.03] 수집 중...
[2024.02.04] 수집 중...
[2024.02.05] 수집 중...
[2024.02.06] 수집 중...
[2024.02.07] 수집 중...
[2024.02.08] 수집 중...
[2024.02.09] 수집 중...
[2024.02.10] 수집 중...
[2024.02.11] 수집 중...
[2024.02.12] 수집 중...
[2024.02.13] 수집 중...
[2024.02.14] 수집 중...
[2024.02.15] 수집 중...
[2024.02.16] 수집 중...
[2024.02.17] 

In [1]:
# 기사 본문 저장
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm 

df = pd.read_csv("QQQ2022_urls.csv", encoding="utf-8")

results = []

headers = {
    "User-Agent": "Mozilla/5.0"
}

for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
    url = row["url"]
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")

        title_tag = soup.select_one("h2#title_area")
        content_tag = soup.select_one("article#dic_area")

        title = title_tag.get_text(strip=True) if title_tag else ""
        content = content_tag.get_text(strip=True) if content_tag else ""

        results.append({
            "company": row["company"],
            "date": row["date"],
            "url": url,
            "title": title,
            "text": content
        })

    except Exception as e:
        print(f"Error processing {url}: {e}")

try:
    pd.DataFrame(results).to_csv("QQQ2022_body.csv", encoding="utf-8-sig", index=False)
    print("저장완료")
except Exception as e:
    print("저장 실패:", e)


Processing: 100%|████████████████████████████████████████████████████████████████████| 869/869 [04:49<00:00,  3.01it/s]

저장완료


---------------------------------------------

In [1]:
# 기사 본문 저장(현우님 파일)
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

df = pd.read_csv("news_links_2023.csv", encoding="utf-8", nrows=100) # 100개 수집
results = []

headers = {
    "User-Agent": "Mozilla/5.0"
}

for index, row in tqdm(df.iterrows(), total=len(df), desc="뉴스 수집 진행 중"):
    url = row["url"]
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")

        title_tag = soup.select_one("h2#title_area")
        content_tag = soup.select_one("article#dic_area")

        title = title_tag.get_text(strip=True) if title_tag else ""
        content = content_tag.get_text(strip=True) if content_tag else ""

        results.append({
            "date": row["date"],
            "url": url,
            "title": title,
            "text": content
        })

    except Exception as e:
        print(f"Error processing {url}: {e}")

# 저장
try:
    pd.DataFrame(results).to_csv("naverNews_body2.csv", encoding="utf-8-sig", index=False)
    print("저장완료")
except Exception as e:
    print("저장 실패:", e)


뉴스 수집 진행 중: 100%|█████████████████████████████████████████████████████████████| 100/100 [00:44<00:00,  2.23it/s]

저장완료
